In [1]:
#!pip install timm
#!pip install ipywidgets

# new model - coatnet
hybrid architecture - Convolution + Attention.

In [2]:
import torch
import numpy as np
import evaluate
from datasets import load_from_disk
import timm
from transformers import (
    TrainingArguments, 
    Trainer
)
from torchvision.transforms import (
    Compose, Resize, CenterCrop, ToTensor, Normalize, 
    RandomHorizontalFlip, RandomResizedCrop
)

DATA_PATH = "processed_bird_data"
OUTPUT_DIR = "new_model_checkpoints"

# Using 'coatnet_0_rw_224' model, with '0' to use the smallest version to avoid overfitting:
MODEL_NAME = "coatnet_0_rw_224"

Applying Aggressive Data Augmentation to prevent overfitting:

1. Does random resize/zoom (scale 0.8–1.0). Forces the model to recognize a bird by looking at its specific feature (e.g. the head, wing, etc.) instead of the background (e.g. tress).
2. Incorporates horizontal flipping to double the training data.

In [3]:
print("Loading dataset.")
dataset = load_from_disk(DATA_PATH)

# transformations:
# normalization of values for ImageNet:
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# training: random crops + flips (noise helps it learn):
_train_transforms = Compose([
    RandomResizedCrop(224, scale=(0.8, 1.0)), 
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

# validation - deterministic center crop
_val_transforms = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    normalize,
])

def train_transforms(batch):
    batch["pixel_values"] = [_train_transforms(image.convert("RGB")) for image in batch["image"]]
    return batch

def val_transforms(batch):
    batch["pixel_values"] = [_val_transforms(image.convert("RGB")) for image in batch["image"]]
    return batch

# using .map() instead of .set_transform() because set_transform keeps the original columns accessible, and we want to delete those raw pngs to avoid program crashing.
print("Applying transforms and removing raw images...")

dataset["train"] = dataset["train"].map(
    train_transforms, 
    batched=True, 
    remove_columns=["image"]
)

dataset["validation"] = dataset["validation"].map(
    val_transforms, 
    batched=True, 
    remove_columns=["image"]
)

print("Data ready.")

Loading dataset.
Applying transforms and removing raw images...
Data ready.


# Training
Initializing CoAtNet model with random weights, no pretraining. 

It uses standard Convolutional layers in the early stages to extract low-level features (edges, textures), and then uses Transformer layers in the final stages to understand the global shape of the bird. Also:

1. High learning rate;
2. 15 epochs;
3. Uses regularization to prevent the model from memorizing the training images.

(using TimmTrainer class as the timm library and Hugging Face Trainer have different variable names (pixel_values vs x)).

In [4]:
import sys

# using the 'timm' library for CoAtNet: update: maybe not
model = timm.create_model(
    MODEL_NAME, 
    pretrained=False, 
    num_classes=200 
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

class TimmTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epoch_train_loss = 0.0
        self.epoch_train_acc = 0.0
        self.epoch_steps = 0

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        pixel_values = inputs.get("pixel_values")
        labels = inputs.get("labels")
        
        logits = model(pixel_values)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        
        if model.training:
            with torch.no_grad():
                preds = torch.argmax(logits, dim=-1)
                acc = (preds == labels).float().mean().item()
                
                self.epoch_train_loss += loss.item()
                self.epoch_train_acc += acc
                self.epoch_steps += 1
                
                if self.epoch_steps % 20 == 0:
                    current_epoch_float = self.state.epoch if self.state.epoch is not None else 0
                    print(f" >> Epoch: {current_epoch_float:.2f} | Batch: {self.epoch_steps} | Curr Loss: {loss.item():.4f}")
        
        return (loss, logits) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        pixel_values = inputs.get("pixel_values")
        labels = inputs.get("labels")
        with torch.no_grad():
            logits = model(pixel_values)
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        if prediction_loss_only:
            return (loss, None, None)
        return (loss, logits, labels)

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        
        avg_train_loss = self.epoch_train_loss / self.epoch_steps if self.epoch_steps > 0 else 0
        avg_train_acc = self.epoch_train_acc / self.epoch_steps if self.epoch_steps > 0 else 0
        
        val_loss = metrics.get(f"{metric_key_prefix}_loss", 0.0)
        val_acc = metrics.get(f"{metric_key_prefix}_accuracy", 0.0)
        
        epoch_num = int(self.state.epoch) if self.state.epoch else 0
        
        print("\n" + "="*80)
        print(f" Training Loss:   {avg_train_loss:.4f} | Training Acc:   {avg_train_acc*100:.2f}%")
        print(f" Validation Loss: {val_loss:.4f}       | Validation Acc: {val_acc*100:.2f}%")
        print("="*80 + "\n")
        
        # resetting counters
        self.epoch_train_loss = 0.0
        self.epoch_train_acc = 0.0
        self.epoch_steps = 0
        
        return metrics

# accuracy metrics:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# training 
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=32, 
    num_train_epochs=30,
    learning_rate=5e-4,
    weight_decay=0.05,
    warmup_ratio=0.1,
    
    disable_tqdm=True, # for now bc i dont like it - Julia
    
    logging_strategy="epoch", 
    save_strategy="epoch",
    eval_strategy="epoch",
    
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to="none"
)

# the actual training
trainer = TimmTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("final_new_model")
print("Best model saved.")

/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 0.18 | Batch: 20 | Curr Loss: 5.2920
 >> Epoch: 0.37 | Batch: 40 | Curr Loss: 5.2192
 >> Epoch: 0.56 | Batch: 60 | Curr Loss: 5.2060
 >> Epoch: 0.75 | Batch: 80 | Curr Loss: 4.8901
 >> Epoch: 0.94 | Batch: 100 | Curr Loss: 4.9773
{'loss': 5.2121, 'grad_norm': 10.421049118041992, 'learning_rate': 0.0001650793650793651, 'epoch': 1.0}
{'eval_loss': 5.13273286819458, 'eval_accuracy': 0.015280135823429542, 'eval_runtime': 35.7113, 'eval_samples_per_second': 16.493, 'eval_steps_per_second': 2.072, 'epoch': 1.0}

 Training Loss:   5.2121 | Training Acc:   2.23%
 Validation Loss: 5.1327       | Validation Acc: 1.53%



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 1.18 | Batch: 20 | Curr Loss: 4.7737
 >> Epoch: 1.37 | Batch: 40 | Curr Loss: 4.9461
 >> Epoch: 1.56 | Batch: 60 | Curr Loss: 4.8216
 >> Epoch: 1.75 | Batch: 80 | Curr Loss: 4.9474
 >> Epoch: 1.94 | Batch: 100 | Curr Loss: 4.9641
{'loss': 4.9344, 'grad_norm': 9.2814302444458, 'learning_rate': 0.00033174603174603175, 'epoch': 2.0}
{'eval_loss': 4.870268821716309, 'eval_accuracy': 0.035653650254668934, 'eval_runtime': 34.3857, 'eval_samples_per_second': 17.129, 'eval_steps_per_second': 2.152, 'epoch': 2.0}

 Training Loss:   4.9344 | Training Acc:   2.56%
 Validation Loss: 4.8703       | Validation Acc: 3.57%



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 16.18 | Batch: 20 | Curr Loss: 0.2428
 >> Epoch: 16.37 | Batch: 40 | Curr Loss: 0.1354
 >> Epoch: 16.56 | Batch: 60 | Curr Loss: 0.3285
 >> Epoch: 16.75 | Batch: 80 | Curr Loss: 0.3511
 >> Epoch: 16.94 | Batch: 100 | Curr Loss: 0.4046
{'loss': 0.2256, 'grad_norm': 11.726457595825195, 'learning_rate': 0.00024091710758377424, 'epoch': 17.0}
{'eval_loss': 4.2865800857543945, 'eval_accuracy': 0.21731748726655348, 'eval_runtime': 34.649, 'eval_samples_per_second': 16.999, 'eval_steps_per_second': 2.136, 'epoch': 17.0}

 Training Loss:   0.2256 | Training Acc:   95.70%
 Validation Loss: 4.2866       | Validation Acc: 21.73%



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 17.18 | Batch: 20 | Curr Loss: 0.2826


KeyboardInterrupt: 

In [ ]:
# test